In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style()

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os, sys, time
import cPickle as pickle
sys.path.append('../EXP/')
import ClevelandMcGill as C
reload(C)

<module 'ClevelandMcGill' from '../EXP/ClevelandMcGill/__init__.pyc'>

In [3]:
NOISE = True

In [6]:
train_target = 6000
val_target = 2000
test_target = 2000

X_train = np.zeros((train_target, 100, 100), dtype=np.float32)
y_train = np.zeros((train_target), dtype=np.float32)
train_counter = 0

X_val = np.zeros((val_target, 100, 100), dtype=np.float32)
y_val = np.zeros((val_target), dtype=np.float32)
val_counter = 0

X_test = np.zeros((test_target, 100, 100), dtype=np.float32)
y_test = np.zeros((test_target), dtype=np.float32)
test_counter = 0

t0 = time.time()

all_counter = 0
while train_counter < train_target or val_counter < val_target or test_counter < test_target:
  
  all_counter += 1
  
  sparse, image, label, parameters = C.Figure1.shading()
  
  if label == 0:
    break
  
  # we need float
  image = image.astype(np.float32)
  
  pot = np.random.choice(3, p=([.6,.2,.2]))

  if pot == 0 and train_counter < train_target:
    # a training candidate
    if label in y_val or label in y_test:
      # no thank you
      continue
      
    # add noise?
    if NOISE:
      image += np.random.uniform(0, 0.05,(100,100))
      
    # safe to add to training
    X_train[train_counter] = image
    y_train[train_counter] = label
    train_counter += 1
    
  elif pot == 1 and val_counter < val_target:
    # a validation candidate
    if label in y_train or label in y_test:
      # no thank you
      continue
      
    # add noise?
    if NOISE:
      image += np.random.uniform(0, 0.05,(100,100))
      
    # safe to add to validation
    X_val[val_counter] = image
    y_val[val_counter] = label
    val_counter += 1
  
  elif pot == 2 and test_counter < test_target:
    # a test candidate
    if label in y_train or label in y_val:
      # no thank you
      continue
      
    # add noise?
    if NOISE:
      image += np.random.uniform(0, 0.05,(100,100))
      
    # safe to add to test
    X_test[test_counter] = image
    y_test[test_counter] = label
    test_counter += 1
  
  
  
print 'Done', time.time()-t0, 'seconds (', all_counter, 'iterations)'

Done 140.554594994 seconds ( 56144 iterations)


In [5]:
X_min = min(X_train.min(), X_val.min(), X_test.min())
X_max = max(X_train.max(), X_val.max(), X_test.max())
y_min = min(y_train.min(), y_val.min(), y_test.min())
y_max = max(y_train.max(), y_val.max(), y_test.max())

# scale in place
X_train -= X_min
X_train /= (X_max - X_min)
X_val -= X_min
X_val /= (X_max - X_min)
X_test -= X_min
X_test /= (X_max - X_min)
y_train -= y_min
y_train /= (y_max - y_min)
y_val -= y_min
y_val /= (y_max - y_min)
y_test -= y_min
y_test /= (y_max - y_min)

# normalize to -.5 .. .5
X_train -= .5
X_val -= .5
X_test -= .5

print 'memory usage', (X_train.nbytes + X_val.nbytes + X_test.nbytes + y_train.nbytes + y_val.nbytes + y_test.nbytes) / 1000000., 'MB'

memory usage 4000.4 MB


In [6]:
X_train_3D = np.stack((X_train,)*3, -1)
X_val_3D = np.stack((X_val,)*3, -1)
X_test_3D = np.stack((X_test,)*3, -1)
print 'memory usage', (X_train_3D.nbytes + X_val_3D.nbytes + X_test_3D.nbytes) / 1000000., 'MB'

memory usage 12000.0 MB


In [7]:
import keras.applications

VGG19 = keras.applications.VGG19(include_top=False, weights='imagenet', input_shape=(100,100,3))

t0 = time.time()
X_train_3D_features = VGG19.predict(X_train_3D, verbose=True)
X_val_3D_features = VGG19.predict(X_val_3D, verbose=True)
X_test_3D_features = VGG19.predict(X_test_3D, verbose=True)
print 'VGG19 features done after', time.time()-t0
print 'memory usage', (X_train_3D_features.nbytes + X_val_3D_features.nbytes + X_test_3D_features.nbytes) / 1000000., 'MB'

Using TensorFlow backend.


20000/20000 [==============================] - 57s 3ms/step
VGG19 features done after 277.631000996
memory usage 1843.2 MB


In [8]:
feature_shape = X_train_3D_features.shape[1] * X_train_3D_features.shape[2] * X_train_3D_features.shape[3]

In [9]:
from keras import models
from keras import layers
from keras import optimizers
import keras.applications
from keras import backend as K
from keras.utils.np_utils import to_categorical

MLP = models.Sequential()

MLP.add(layers.Dense(256, activation='relu', input_dim=feature_shape))
MLP.add(layers.Dropout(0.5))
MLP.add(layers.Dense(1, activation='linear')) # REGRESSION

sgd = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
MLP.compile(loss='mean_squared_error', optimizer=sgd, metrics=['mse', 'mae']) # MSE for regression

In [10]:
t0 = time.time()
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto')]

history = MLP.fit(X_train_3D_features.reshape(X_train_3D_features.shape[0], feature_shape), \
                  y_train, \
                  epochs=1000, \
                  batch_size=32, \
                  validation_data=(X_val_3D_features.reshape(X_val_3D_features.shape[0], feature_shape), y_val),
                  callbacks=callbacks,
                  verbose=True)
print 'Fitting done', time.time()-t0

Train on 60000 samples, validate on 20000 samples
Epoch 1/1000
60000/60000 [==============================] - 9s 152us/step - loss: 0.0531 - mean_squared_error: 0.0531 - mean_absolute_error: 0.1732 - val_loss: 0.0174 - val_mean_squared_error: 0.0174 - val_mean_absolute_error: 0.0984
Epoch 2/1000
60000/60000 [==============================] - 9s 148us/step - loss: 0.0257 - mean_squared_error: 0.0257 - mean_absolute_error: 0.1244 - val_loss: 0.0116 - val_mean_squared_error: 0.0116 - val_mean_absolute_error: 0.0798
Epoch 3/1000
60000/60000 [==============================] - 9s 147us/step - loss: 0.0202 - mean_squared_error: 0.0202 - mean_absolute_error: 0.1094 - val_loss: 0.0089 - val_mean_squared_error: 0.0089 - val_mean_absolute_error: 0.0694
Epoch 4/1000
60000/60000 [==============================] - 9s 147us/step - loss: 0.0174 - mean_squared_error: 0.0174 - mean_absolute_error: 0.1009 - val_loss: 0.0073 - val_mean_squared_error: 0.0073 - val_mean_absolute_error: 0.0611
Epoch 5/1000
6

Epoch 36/1000
60000/60000 [==============================] - 9s 146us/step - loss: 0.0057 - mean_squared_error: 0.0057 - mean_absolute_error: 0.0569 - val_loss: 0.0016 - val_mean_squared_error: 0.0016 - val_mean_absolute_error: 0.0307
Epoch 37/1000
60000/60000 [==============================] - 9s 147us/step - loss: 0.0056 - mean_squared_error: 0.0056 - mean_absolute_error: 0.0564 - val_loss: 0.0015 - val_mean_squared_error: 0.0015 - val_mean_absolute_error: 0.0300
Epoch 38/1000
60000/60000 [==============================] - 9s 146us/step - loss: 0.0055 - mean_squared_error: 0.0055 - mean_absolute_error: 0.0561 - val_loss: 0.0015 - val_mean_squared_error: 0.0015 - val_mean_absolute_error: 0.0294
Epoch 39/1000
60000/60000 [==============================] - 9s 146us/step - loss: 0.0054 - mean_squared_error: 0.0054 - mean_absolute_error: 0.0558 - val_loss: 0.0020 - val_mean_squared_error: 0.0020 - val_mean_absolute_error: 0.0345
Epoch 40/1000
60000/60000 [==============================] -

Epoch 71/1000
60000/60000 [==============================] - 9s 147us/step - loss: 0.0046 - mean_squared_error: 0.0046 - mean_absolute_error: 0.0512 - val_loss: 0.0013 - val_mean_squared_error: 0.0013 - val_mean_absolute_error: 0.0274
Epoch 72/1000
60000/60000 [==============================] - 9s 146us/step - loss: 0.0046 - mean_squared_error: 0.0046 - mean_absolute_error: 0.0511 - val_loss: 0.0015 - val_mean_squared_error: 0.0015 - val_mean_absolute_error: 0.0302
Epoch 73/1000
60000/60000 [==============================] - 9s 147us/step - loss: 0.0046 - mean_squared_error: 0.0046 - mean_absolute_error: 0.0510 - val_loss: 0.0012 - val_mean_squared_error: 0.0012 - val_mean_absolute_error: 0.0267
Epoch 74/1000
60000/60000 [==============================] - 9s 146us/step - loss: 0.0046 - mean_squared_error: 0.0046 - mean_absolute_error: 0.0512 - val_loss: 0.0013 - val_mean_squared_error: 0.0013 - val_mean_absolute_error: 0.0273
Epoch 75/1000
60000/60000 [==============================] -

Epoch 106/1000
60000/60000 [==============================] - 9s 147us/step - loss: 0.0042 - mean_squared_error: 0.0042 - mean_absolute_error: 0.0490 - val_loss: 0.0010 - val_mean_squared_error: 0.0010 - val_mean_absolute_error: 0.0248
Epoch 107/1000
60000/60000 [==============================] - 9s 146us/step - loss: 0.0042 - mean_squared_error: 0.0042 - mean_absolute_error: 0.0489 - val_loss: 9.9739e-04 - val_mean_squared_error: 9.9739e-04 - val_mean_absolute_error: 0.0245
Epoch 108/1000
60000/60000 [==============================] - 9s 147us/step - loss: 0.0042 - mean_squared_error: 0.0042 - mean_absolute_error: 0.0487 - val_loss: 0.0011 - val_mean_squared_error: 0.0011 - val_mean_absolute_error: 0.0254
Epoch 109/1000
60000/60000 [==============================] - 9s 144us/step - loss: 0.0042 - mean_squared_error: 0.0042 - mean_absolute_error: 0.0485 - val_loss: 0.0011 - val_mean_squared_error: 0.0011 - val_mean_absolute_error: 0.0250
Epoch 110/1000
60000/60000 [====================

In [11]:
y_pred = MLP.predict(X_test_3D_features.reshape(X_test_3D_features.shape[0], feature_shape))

In [12]:
import sklearn.metrics
np.log2(sklearn.metrics.mean_absolute_error(y_pred*100, y_test*100)+.125)

1.5910425575862956

In [13]:
y_pred

array([[ 0.22938162],
       [ 0.14699593],
       [ 0.6785295 ],
       ..., 
       [ 0.71962476],
       [ 0.86336124],
       [ 0.2867682 ]], dtype=float32)

In [14]:
y_test

array([ 0.25842696,  0.14606741,  0.75280899, ...,  0.75280899,
        0.86516851,  0.29213482], dtype=float32)

In [15]:
sklearn.metrics.mean_absolute_error(y_pred*100, y_test*100)

2.8876698